In [ ]:
import numpy as np

def histogram_equalization_2d(arr: np.ndarray) -> np.ndarray:
    """
    對 2D 浮點數陣列做直方圖均衡化。
    輸入值域應在 0~255(不必為整數) 輸出會轉成同樣值域的浮點數。

    Parameters:
        arr (np.ndarray): shape=(H, W) 值域在 0~255 的 2D 浮點數陣列

    Returns:
        np.ndarray: 直方圖均衡化後的 2D 陣列) float64, 值域仍為 0~255
    """
    flat = arr.flatten()
    
    # 轉為整數 bins（0~255）做 histogram 分類
    hist, bins = np.histogram(flat, bins=256, range=(0, 255))
    cdf = hist.cumsum()
    cdf_normalized = cdf / cdf[-1]  # Normalize to [0,1]

    # 建立映射表
    equalized_map = np.interp(flat, bins[:-1], cdf_normalized * 255)

    # reshape 回原來形狀
    return equalized_map.reshape(arr.shape)

In [18]:
from scipy.ndimage import zoom

def adaptive_histogram_equalization_2d(arr: np.ndarray, tile_size=32, clip_limit=None) -> np.ndarray:
    """
    對 2D 浮點數陣列進行自適應直方圖均衡化 (AHE)。
    每個 tile 區塊進行 local histogram equalization, 再使用雙線性插值合併結果。

    Parameters:
        arr (np.ndarray): 2D array, 值域為 0~255 的浮點數
        tile_size (int): tile 區塊的大小 (建議 8~64)
        clip_limit (float or None): 若提供則做簡易版 CLAHE, 限制每個 bin 的最大頻率

    Returns:
        np.ndarray: 均衡化後的陣列, shape 與 arr 相同, dtype=float64, 值域為 0~255
    """
    h, w = arr.shape
    arr = np.clip(arr, 0, 255)
    
    # 分割成 tile 的數量
    n_tiles_y = int(np.ceil(h / tile_size))
    n_tiles_x = int(np.ceil(w / tile_size))
    
    # 儲存每個 tile 處理後的結果
    processed_tiles = np.zeros((n_tiles_y, n_tiles_x, tile_size, tile_size))

    for ty in range(n_tiles_y):
        for tx in range(n_tiles_x):
            y0 = ty * tile_size
            y1 = min((ty + 1) * tile_size, h)
            x0 = tx * tile_size
            x1 = min((tx + 1) * tile_size, w)
            
            tile = arr[y0:y1, x0:x1]
            flat = tile.flatten()
            hist, bins = np.histogram(flat, bins=256, range=(0, 255))

            # Apply clip limit (簡化版 CLAHE)
            if clip_limit is not None:
                excess = hist - clip_limit
                excess[excess < 0] = 0
                hist = np.minimum(hist, clip_limit)
                hist += excess.sum() // 256  # 均分回去

            cdf = hist.cumsum()
            cdf = cdf / cdf[-1]  # Normalize to [0, 1]

            # 映射並還原形狀
            mapped = np.interp(flat, bins[:-1], cdf * 255)
            enhanced_tile = mapped.reshape(tile.shape)

            # 存入 (pad 到 tile_size)
            full_tile = np.zeros((tile_size, tile_size))
            full_tile[:tile.shape[0], :tile.shape[1]] = enhanced_tile
            processed_tiles[ty, tx] = full_tile

    # 組合所有 tile：先重組為大矩陣，再裁切
    combined = np.block([[processed_tiles[ty, tx] for tx in range(n_tiles_x)] for ty in range(n_tiles_y)])
    return combined[:h, :w]


In [43]:
from skimage import exposure

def clip_limit_adaptive_histogram_equalization(arr):
    # arr 是 0~255 的 2D float array
    img = arr / 255.0  # 轉成 0~1
    clahe_img = exposure.equalize_adapthist(img, clip_limit=0.01)
    clahe_img = (clahe_img * 255).astype(np.uint8)
    return clahe_img

In [1]:
import cv2

def opencv_clahe(arr):
    arr = np.clip(arr, 0, 255)  # 確保值域合法
    arr_uint8 = arr.astype(np.uint8)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(arr_uint8)
    return enhanced

In [2]:
import numpy as np
from PIL import Image

def array_to_grayscale_image(arr: np.ndarray) -> Image.Image:
    """
    將 2D 浮點陣列轉換成灰階圖像，每個值對應一個 pixel，顏色為 floor(value)。

    Parameters:
        arr (np.ndarray): 2D float array，值域應為 [0, 255]

    Returns:
        PIL.Image: 對應的灰階圖像 (mode='L')
    """
    arr = np.clip(arr, 0, 255)           # 限制值域
    int_arr = np.floor(arr).astype(np.uint8)  # 轉為 uint8 整數值
    return Image.fromarray(int_arr, mode='L')


In [3]:
def new_w_and_h(old_w, old_h, value):
    if IS_WIDTH:
        new_width = value
        new_height = int(old_h * FONT_BBOX_RATIO * new_width / old_w)
        return (new_width, new_height)
    # else
    new_height = value
    new_width = int(old_w * new_height / (old_h * FONT_BBOX_RATIO))
    return (new_width, new_height)

def block_average_downsample(image, char_length):
    # 讀取圖片
    width, height = image.size
    pixels = np.array(image, dtype=np.float32)

    # 計算每個新 pixel 對應原圖的區塊大小
    new_width, new_height = new_w_and_h(width, height, char_length)
    block_w = width / new_width
    block_h = height / new_height

    # 初始化輸出陣列
    downsampled = np.zeros((new_height, new_width), dtype=np.float32)

    for y in range(new_height):
        for x in range(new_width):
            # 對應的原圖區塊範圍
            x_start = int(x * block_w)
            x_end = int((x + 1) * block_w)
            y_start = int(y * block_h)
            y_end = int((y + 1) * block_h)

            block = pixels[y_start:y_end, x_start:x_end]
            downsampled[y, x] = np.mean(block)

    return downsampled  # 每個值為 float 灰階 0~255

In [4]:
from PIL import Image

def str_to_bool(s):
    return s.lower() in ("true", "1", "yes", "y")

# file_name = input("file_name: ")
# VSCODE = str_to_bool(input("VSCODE: "))
# char_length = int(input("char_length: "))
# IS_WIDTH = input("IS_WIDTH: ") == "w"

file_name = "cry2.jpg"
VSCODE = True
char_length = 256
IS_WIDTH = True
image = Image.open("C:/Users/zyqio/source/repos/SidePro/pics/" + file_name).convert("L")

# FONT_BBOX_RATIO = 0.455 if VSCODE else 0.44
FONT_BBOX_RATIO = 1
resized = block_average_downsample(image, char_length)

In [34]:
img_origin = array_to_grayscale_image(resized)
img_origin.save("img_origin.png")

In [35]:
HE = histogram_equalization_2d(resized)
img_HE = array_to_grayscale_image(HE)
img_HE.save("img_HE.png")

In [45]:
AHE = adaptive_histogram_equalization_2d(resized, 32, 3.0)
img_AHE = array_to_grayscale_image(AHE)
img_AHE.save("img_AHE.png")

In [44]:
CLAHE = clip_limit_adaptive_histogram_equalization(resized)
image_CLAHE = array_to_grayscale_image(CLAHE)
image_CLAHE.save("img_CLAHE.png")

In [5]:
CV = opencv_clahe(resized)
image_CV = array_to_grayscale_image(CV)
image_CV.save("img_CV.png")